In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110570

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,39838.1084
2018-02-28,35433.0663
2018-03-31,35546.6970
2018-04-30,39209.0735
2018-05-31,37356.9896
2018-06-30,33332.2938
2018-07-31,43120.4723
2018-08-31,37425.3603
2018-09-30,41079.3484


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    39838.1084
 2018-02-28    35433.0663
 2018-03-31    35546.6970
 2018-04-30    39209.0735
 2018-05-31    37356.9896
 2018-06-30    33332.2938
 2018-07-31    43120.4723
 2018-08-31    37425.3603
 2018-09-30    41079.3484
 2018-10-31    36709.7217
 2018-11-30    32020.1000
 2018-12-31    31472.7355
 2019-01-31    45212.9242
 2019-02-28    37886.0526
 2019-03-31    46727.8800
 2019-04-30    38133.6778
 2019-05-31    45692.2199
 2019-06-30    53121.5098
 2019-07-31    50063.3045
 2019-08-31    52130.8279
 2019-09-30    50274.5622
 2019-10-31    46092.8228
 2019-11-30    55255.4703
 2019-12-31    51392.9522
 2020-01-31    50170.9247
 2020-02-29    48834.7359
 2020-03-31    29404.4184
 2020-04-30    32233.8416
 2020-05-31    38505.1081
 2020-06-30    36994.2583
 2020-07-31    34773.6388
 2020-08-31    34104.2546
 2020-09-30    30306.5435
 2020-10-31    32415.2525
 2020-11-30    30670.3536
 2020-12-31    32189.2969
 2021-01-31    29965.2791
 2021-02-

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.026252
p-value : 0.275183
Critical Values :
	1%: -3.632743
	5%: -2.948510
	10%: -2.613017


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -9.128380
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=873.885, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=863.348, Time=0.12 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=860.396, Time=0.07 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=849.362, Time=0.18 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=863.493, Time=0.08 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=863.080, Time=0.13 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=846.161, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=845.447, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=847.351, Time=0.06 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=849.608, Time=0.07 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=851.592, Time=0.12 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=845.209, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=847.141, Time=0.07 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=849.382, Time=0.14 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=851.477, Time=0.17 sec
 ARIMA(3,0,0)(0,0,0)[1] i

ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(2,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(2, 0)   Log Likelihood                -418.569
Method:                       css-mle   S.D. of innovations           5101.351
Date:                Sat, 11 Sep 2021   AIC                            845.139
Time:                        04:05:47   BIC                            852.089
Sample:                             0   HQIC                           847.687
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.832e+04   3235.744     11.844      0.000     3.2e+04    4.47e+04
ar.L1.y        0.5135      0.147      3.497      0.000       0.226       0.801
ar.L2.y        0.2658      0.147      1.804      0.0

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([34129.82988439, 34576.15367309]), array([5101.3506699 , 5734.62847731]), array([[24131.36629888, 44128.29346989],
       [23336.48839285, 45815.81895334]]))
